This is a task for The Eluvio about Data Science/ML.  

In [1]:
# Imports
import torch
import numpy as np
import torchtext.legacy
import pandas as pd
import re
import string
import random
import os 
import torch.nn as nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Upload File/s
from google.colab import drive
drive.mount("/content/gdrive")

# uploading from computer
from google.colab import files
uploaded = files.upload()

Mounted at /content/gdrive


In [ ]:
# Data Read
data = pd.read_csv('/content/gdrive/MyDrive/Eluvio/Eluvio_DS_Challenge.csv', sep=',', error_bad_lines=False)
separated_data = data[["title", "up_votes"]]
stop_words = set(stopwords.words("english"))# getting the stopwords
separated_data.dropna(subset = ["title"], inplace=True)
separated_data.dropna(subset = ["up_votes"], inplace=True)
separated_data['title'] = separated_data['title'].str.replace('[{}]'.format(string.punctuation), ' ')
separated_data['title'] = separated_data['title'].str.replace('[{}]'.format('।'), '')
# separated_data = separated_data.head(100)
separated_data['title'] = separated_data['title'].apply(lambda x: ' '.join([item for item in x.split() if item not in (stop_words)]))

In [4]:
# Lemmatization function
lemma = WordNetLemmatizer()
def lematization(text):
  word_list = nltk.word_tokenize(text)
  # Lemmatization
  text = [lemma.lemmatize(w) for w in word_list]
  # removing words lenth less 2
  text = [i for i in text if len(i)>2]
  # bring back to normal text
  text = " ".join(text)
  
  return text

In [5]:
#Dataset Cleanup
for i in range(len(separated_data)):
    #Lower Case
    text = separated_data["title"][i].lower()
    # remove punctuation
    text = re.sub('[^a-zA-Z]', ' ', text)
    # remove links
    text= re.sub(r' http\S+', ' ', text)
    # lematization
    text = lematization(text)
    # remove newline
    text = re.sub(r'\n', '', text)
    # remove multiple spaces
    separated_data["title"][i] = re.sub(r' +', ' ', text)
    
separated_data.drop(separated_data.index[np.where(separated_data.applymap(lambda x: x == ''))[0]], inplace=True) # Removing empty strings 
separated_data = separated_data.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
# using the viewers up votes to decide about the title acceptance
avg = separated_data["up_votes"].mean()
for i in range(len(separated_data["up_votes"])):
  if separated_data["up_votes"][i]>avg:
    separated_data["up_votes"][i] = 1.0
  else:
    separated_data["up_votes"][i] = 0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
"""we can even use different quartiles to create classes"""

# quantiles = separated_data.up_votes.quantile([0.25,0.5,0.75]).values
# first_quantile = quantiles[0]
# second_quantile = quantiles[1]
# third_quantile = quantiles[2]


'we can even use different quartiles to create classes'

In [17]:
# training and testing data are separated
training_data = separated_data.sample(frac = 0.8)
testing_data = separated_data.drop(training_data.index)
# Training data save
training_data.to_csv("training_data.tsv", sep="\t", index=False)
# testing data save
testing_data.to_csv("testing_data.tsv", sep="\t", index=False)

#For Creating Train and Validation data
char_based = False
if char_based:
    tokenizer = lambda s: list(s) # char-based
else:
    tokenizer = lambda s: s.split() # word-based

Text = torchtext.legacy.data.Field(tokenize=tokenizer, batch_first=True, include_lengths=True, fix_length=100)
Label = torchtext.legacy.data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

fields = [('text', Text), ('labels', Label)]
training_data, testing_data = torchtext.legacy.data.TabularDataset.splits(
    path = "/content",
    train = "training_data.tsv",
    test = "testing_data.tsv",
    format='tsv',
    fields=fields,
    skip_header=True
)

seed = 42

# train and validation set
training_data, validation_data = training_data.split(split_ratio=0.8, random_state=random.seed(seed))

In [18]:
# Functions for training, testing and evaluation
def create_iterator(train_data, valid_data, test_data, batch_size, device):    
  train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits((train_data, valid_data, test_data),
          batch_size = batch_size,
          sort_key = lambda x: len(x.text), 
          sort_within_batch = True,
          device = device)
  return train_iterator, valid_iterator, test_iterator

def accuracy(probs, target):
  predictions = probs.argmax(dim=1)
  corrects = (predictions == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text, text_lengths = text.to(device), text_lengths.to(device)
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.labels.squeeze())
        acc = accuracy(predictions, batch.labels)        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.labels)
            acc = accuracy(predictions, batch.labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def run_train(epochs, model, train_iterator, valid_iterator, optimizer, criterion, model_type):
  best_valid_loss = float('inf')
  training_loss_list = []
  valid_loss_list = []
  for epoch in range(epochs):

      # train the model
      train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

      # evaluate the model
      valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

      # save the best model
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'saved_weights'+'_'+model_type+'.pt')
      
      print(f'Epoch: {epoch+1} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
      print(f'Epoch: {epoch+1} | Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

In [25]:
# Hyperparameters
lr = 1e-5
batch_size = 50
dropout_keep_prob = 0.9
embedding_size = 300
max_document_length = 100  # each sentence has until 100 words
dev_size = 0.8 # split percentage to train\validation data
seed = 1
num_classes = 2 # if we increase the number of classes, we have to give that value here.
num_hidden_nodes = 93
hidden_dim2 = 128
num_layers = 2  # LSTM layers
bi_directional = False 
num_epochs = 5

In [26]:
# model
class LSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, n_layers,
                bidirectional, dropout, pad_index):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_index)
      self.lstm = nn.LSTM(embedding_dim,
                          hidden_dim1,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          batch_first=True)
      self.fc1 = nn.Linear(hidden_dim1 * 2, hidden_dim2)
      self.fc2 = nn.Linear(hidden_dim2, output_dim)
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(dropout)

  def forward(self, text, text_lengths):
      embedded = self.embedding(text)
      packed_embedded = pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True) 

      packed_output, (hidden, cell) = self.lstm(packed_embedded)
      cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
      rel = self.relu(cat)
      dense1 = self.fc1(rel)
      drop = self.dropout(dense1)
      preds = self.fc2(drop)
      return preds

In [ ]:
# Main Function
if __name__ == "__main__":
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
  path = '/content'
  path_data = os.path.join(path, "data")
  # parameters
  model_type = "LSTM"
  data_type = "token" # or: "morph"

  char_based = False
  if char_based:
      tokenizer = lambda s: list(s) # char-based
  else:
      tokenizer = lambda s: s.split() # word-based

  Text.build_vocab(training_data,min_freq=2)
  Label.build_vocab(training_data)
  vocab_size = len(Text.vocab)
  # to_train = True
  pad_index = Text.vocab.stoi[Text.pad_token]


  train_iterator, valid_iterator, test_iterator = create_iterator(training_data, validation_data, testing_data, batch_size, device)

  # loss function
  loss_func = nn.CrossEntropyLoss()
  loss_func = loss_func.to(device)
  lstm_model = LSTM(vocab_size, embedding_size, num_hidden_nodes, hidden_dim2 , num_classes, num_layers, bi_directional, dropout_keep_prob, pad_index)
  if torch.cuda.is_available():
    lstm_model.cuda()

  # optimization algorithm
  optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

  torch.backends.cudnn.enabled = False
  run_train(num_epochs, lstm_model, train_iterator, valid_iterator, optimizer, loss_func, model_type)

  test_loss, test_acc = evaluate(lstm_model, test_iterator, loss_func)
  print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')